In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning2

from source.experiment_interface import run_exp_iter_with_LFR

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'LFR_acs_income'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp', 'LFR', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'LFR', EXPERIMENT_NAME, 'folk_GA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '3855ad0b-4d2e-4baf-a5a8-6e3e92f624ba',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  3855ad0b-4d2e-4baf-a5a8-6e3e92f624ba


## Initialize custom objects

In [9]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [10]:
data_loader.X_data.shape

(15000, 10)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [12]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      with_tuning=True,
                      # with_tuning=False,
                      # tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 16:59:40 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "[{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': "
                                 '50.0}]',
 'model_init_seed': 100,
 'session_uuid': '3855ad0b-4d2e-4baf-a5a8-6e3e92f624ba'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}
step: 0, loss: 0.8499657311576877, L_x: 0.5355271319233583,  L_y: 0.7269840436424602,  L_z: 0.002352528323919878
step: 250, loss: 0.8499657593510387, L_x: 0.5355271362189717,  L_y: 0.7269840593420802,  L_z: 0.002352528572935375
step: 500, loss: 0.8499657625541241, L_x: 0.5355271402008388,  L_y: 0.7269840613992492,  L_z: 0.0023525285950573294
step: 750, loss: 0.8499656769902526, L_x: 0.5355271382640682,  L_y: 0.7269840224093211,  L_z: 0.0023525276639658143
step: 1000, loss: 0.8499656934359953, L_x: 0.5355271346477161,  L_y: 0.7269840284398312,  L_z: 0.0023525278729937373
step: 1250, loss: 0.8499656805288781, L_x: 0.5355271365265548,  L_y: 0.7269840227480528,  L_z: 0.0023525277283111936
step: 1500, loss: 0.8499657234703039, L_x: 0.5355271368499446,  L_y: 0.7269840277623107,  L_z: 0.002352528486789873
step: 1750, loss: 0.8499657298564315, L_x: 0.5355271378903022,  L_y: 0.7269840411072381,  L_z: 0.0023525283474058077
step: 

2024-01-02 17:09:23 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=723)
Top indexes of an X_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
2024/01/02, 17:09:23: Tuning MLPClass

2024-01-02 17:12:34 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [11]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_20240102__221234.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [12]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 17:29:58 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "[{'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': "
                                 '50.0}]',
 'model_init_seed': 200,
 'session_uuid': '3855ad0b-4d2e-4baf-a5a8-6e3e92f624ba'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 5, 'Ax': 0.01, 'Ay': 1.0, 'Az': 50.0}
step: 0, loss: 0.8577617167334382, L_x: 0.5356905838047069,  L_y: 0.7262581054267125,  L_z: 0.00252293410937357
step: 250, loss: 0.8577617475857187, L_x: 0.5356905899794714,  L_y: 0.7262581210381541,  L_z: 0.0025229344129553966
step: 500, loss: 0.8577617506835553, L_x: 0.5356905899630687,  L_y: 0.7262581237368737,  L_z: 0.00252293442094102
step: 750, loss: 0.857761672866654, L_x: 0.5356905872493759,  L_y: 0.7262580808956445,  L_z: 0.002522933721970316
step: 1000, loss: 0.8577616884295499, L_x: 0.535690588651478,  L_y: 0.7262580894861851,  L_z: 0.002522933861137
step: 1250, loss: 0.8577616757326948, L_x: 0.5356905891310524,  L_y: 0.7262580831265856,  L_z: 0.0025229337342959736
step: 1500, loss: 0.8577617054475044, L_x: 0.535690588636824,  L_y: 0.7262580919034876,  L_z: 0.0025229341531529726
step: 1750, loss: 0.8577617143595597, L_x: 0.535690588477755,  L_y: 0.7262581029697011,  L_z: 0.002522934110101621
step: 2000, loss:

2024-01-02 17:39:33 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 17:39:33 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=729)
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Path for tuned params:  /home/dh3553/

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [23]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_20240102__221234.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [24]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 14:11:34 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 300,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7979782676913761, L_x: 1.0323907659454599,  L_y: 0.6781985204253934,  L_z: 0.00827033533571835
step: 250, loss: 0.797978199748413, L_x: 1.0323908019191732,  L_y: 0.6781984500730722,  L_z: 0.008270334741711762
step: 500, loss: 0.7979782787217018, L_x: 1.0323907832154928,  L_y: 0.6781985335275081,  L_z: 0.00827033343632219
step: 750, loss: 0.6475774505477387, L_x: 1.0322746489837868,  L_y: 0.5280085073128342,  L_z: 0.008170739168262924
step: 1000, loss: 0.6475774873926531, L_x: 1.0322746589152256,  L_y: 0.5280085418657529,  L_z: 0.008170739817688817
step: 1250, loss: 0.5396801755981412, L_x: 1.0305529553397612,  L_y: 0.42116019193809506,  L_z: 0.007732344063034973
step: 1500, loss: 0.5396801895434374, L_x: 1.0305529807322427,  L_y: 0.42116019036751473,  L_z: 0.007732350551349179
step: 1750, loss: 0.5361632824016606, L_x: 1.0293364091548245,  L_y: 0.41834810283738577,  L_z: 0.007440769324396114
step: 2000, 

2024-01-02 14:11:37 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:11:37 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [25]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_20240102__221234.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [26]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 14:13:52 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 400,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7990623753426728, L_x: 1.0328282366486268,  L_y: 0.6788628044478547,  L_z: 0.00845837361497765
step: 250, loss: 0.799062310279977, L_x: 1.0328282720631976,  L_y: 0.6788627364890443,  L_z: 0.008458373292306452
step: 500, loss: 0.7990623867186859, L_x: 1.0328282538498756,  L_y: 0.6788628177803322,  L_z: 0.008458371776683092
step: 750, loss: 0.653315875429133, L_x: 1.0327228893027587,  L_y: 0.5333254837317553,  L_z: 0.008359051383550958
step: 1000, loss: 0.6533159097569131, L_x: 1.032722899285175,  L_y: 0.5333255157988329,  L_z: 0.008359052014781295
step: 1250, loss: 0.5497125316449264, L_x: 1.031007016982998,  L_y: 0.4307364335163241,  L_z: 0.00793769821515125
step: 1500, loss: 0.5497125427775167, L_x: 1.031007042805463,  L_y: 0.43073643312089216,  L_z: 0.007937702688039144
step: 1750, loss: 0.5462092990230252, L_x: 1.029785484764563,  L_y: 0.4278213886820478,  L_z: 0.007704680932260533
step: 2000, loss: 0

2024-01-02 14:13:55 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:13:55 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [27]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_20240102__221234.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [28]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 14:16:10 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 500,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.8000964558093502, L_x: 1.0320650821383393,  L_y: 0.6802994870863546,  L_z: 0.008295230254580854
step: 250, loss: 0.8000963906903515, L_x: 1.032065117326718,  L_y: 0.6802994200786068,  L_z: 0.008295229439536474
step: 500, loss: 0.8000964681341676, L_x: 1.0320650992688587,  L_y: 0.6802995004138729,  L_z: 0.008295228896704438
step: 750, loss: 0.6536690292061383, L_x: 1.031969089471765,  L_y: 0.53409937860604,  L_z: 0.008186370826460966
step: 1000, loss: 0.6536690630394447, L_x: 1.0319690979327079,  L_y: 0.5340994111791351,  L_z: 0.008186371033519452
step: 1250, loss: 0.5498835673882443, L_x: 1.030299316549742,  L_y: 0.43144221627644824,  L_z: 0.007705709728410874
step: 1500, loss: 0.5498835849308982, L_x: 1.0302993430188478,  L_y: 0.43144221903501456,  L_z: 0.007705715796999389
step: 1750, loss: 0.546021331900659, L_x: 1.0291095664929255,  L_y: 0.42834864138975803,  L_z: 0.0073808669308041905
step: 2000, lo

2024-01-02 14:16:14 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:16:14 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [29]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_GA_2018_Income_20240102__221234.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [30]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSIncomeDataset',
                      verbose=True)

2024-01-02 14:18:23 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 600,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.8012308209526325, L_x: 1.032486691591748,  L_y: 0.6816260094522957,  L_z: 0.00817807117058102
step: 250, loss: 0.8012307592510307, L_x: 1.03248672607275,  L_y: 0.6816259452091471,  L_z: 0.00817807071730432
step: 500, loss: 0.8012308321514614, L_x: 1.032486709583512,  L_y: 0.6816260226759697,  L_z: 0.00817806925857021
step: 750, loss: 0.6601098419829545, L_x: 1.0323946558803412,  L_y: 0.5407171178798217,  L_z: 0.00807662925754936
step: 1000, loss: 0.66010987855932, L_x: 1.0323946661335703,  L_y: 0.5407171500610479,  L_z: 0.00807663094245753
step: 1250, loss: 0.5619558254545324, L_x: 1.0308029125968043,  L_y: 0.443589170867648,  L_z: 0.007643181663601961
step: 1500, loss: 0.5619558401906308, L_x: 1.0308029386365294,  L_y: 0.4435891757918725,  L_z: 0.007643185267552722
step: 1750, loss: 0.5585636756338709, L_x: 1.0296490241047005,  L_y: 0.44074622164810573,  L_z: 0.007426275787647554
step: 2000, loss: 0.558

2024-01-02 14:18:26 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:18:26 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]